In [ ]:
%gui qt

In [ ]:
from importlib import reload
from pprint import pprint

import numpy as np

import pyqtgraph as pg
from pyqtgraph.flowchart import Flowchart, Node as pgNode
from pyqtgraph import Qt
from pyqtgraph.Qt import QtGui, QtCore
from pyqtgraph.flowchart import library as fclib

from plottr.data import datadict; reload(datadict)
from plottr.data.datadict import togrid, DataDict, GridDataDict

In [ ]:
def testdata_1d(nvals=11):
    x = np.linspace(0, 10, nvals)
    y = np.cos(x)
    z = np.cos(x)**2
    d = DataDict(
        x = {'values' : x},
        y = {'values' : y, 'axes' : ['x']},
        z = {'values' : z, 'axes' : ['x']},
    )
    return d

def testdata_3d(nx=3, ny=3, nz=3, rand_factor=1):
    x = np.linspace(0, 10, nx)
    y = np.linspace(-5, 5, ny)
    z = np.arange(nz)
    xx, yy, zz = np.meshgrid(x, y, z, indexing='ij')
    dd = np.cos(xx) * np.sin(yy) + rand_factor * np.random.rand(*zz.shape)
    dd2 = np.sin(xx) * np.cos(yy) + rand_factor * np.random.rand(*zz.shape)
    d = DataDict(
        x = dict(values=xx.reshape(-1), unit='mX'),
        y = dict(values=yy.reshape(-1), unit='kY'),
        z = dict(values=zz.reshape(-1), unit='MZ'),
        data = dict(values=dd.reshape(-1), axes=['x', 'y', 'z'], unit='OMFG'),
        more_data = dict(values=dd2.reshape(-1), axes=['x', 'y', 'z'], unit='WTF'),
        different_data = dict(values=dd2.T.reshape(-1), axes=['z', 'y', 'x'], unit='FML')
    )
    return d

In [ ]:
from plottr.node import node; reload(node)
from plottr.node.node import Node

from plottr.node import data_selector; reload(data_selector)
from plottr.node.data_selector import XYSelector, XYSelectorWidget

from plottr.plot import mpl; reload(mpl)
from plottr.plot.mpl import PlotNode, MPLPlotWidget, AutoPlot

Node.raiseExceptions = True
Node.debug = True
XYSelector.debug = True
XYSelectorWidget.debug = True

nodelib = fclib.NodeLibrary()
nodelib.addNodeType(Node, [('Basic')])
nodelib.addNodeType(XYSelector, [('Basic')])
nodelib.addNodeType(PlotNode, [('Plotting')])

fc = Flowchart(terminals={
    'dataIn': {'io': 'in'},
    'dataOut': {'io': 'out'}
})
fc.library = nodelib

selector = fc.createNode('XYSelector')
plot = fc.createNode('Plot')

fc.connectTerminals(fc['dataIn'], selector['dataIn'])
fc.connectTerminals(selector['dataOut'], fc['dataOut'])
fc.connectTerminals(selector['dataOut'], plot['dataIn'])


dialog = QtGui.QDialog()

optsLayout = QtGui.QVBoxLayout()
optsLayout.addWidget(selector.ctrlWidget())

plotWidget = AutoPlot()
plotLayout = QtGui.QVBoxLayout()
plotLayout.addWidget(plotWidget)

mainLayout = QtGui.QHBoxLayout(dialog)
mainLayout.addLayout(optsLayout)
mainLayout.addLayout(plotLayout)
mainLayout.setStretchFactor(optsLayout, 1)
mainLayout.setStretchFactor(plotLayout, 3)

dialog.show()

plot.setPlotWidget(plotWidget)

fc.setInput(dataIn=testdata_3d(21, 21, 21, rand_factor=0.1))

In [ ]:

fc.inputValues(), fc.output()